In [1]:
#default_exp model.model_shop

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
#export
import torch
import peptdeep.model.building_block as building_block
from peptdeep.model.model_interface import ModelInterface
from peptdeep.model.featurize import (
    get_ascii_indices, get_batch_mod_feature
)
import pandas as pd
import numpy as np

ASCII_NUM=128

# Scalar Regression Models for a Given Amino Acid Sequence

In [4]:
#export

class ScalarRegression_LSTM_Model_for_AASeq(torch.nn.Module):
    def __init__(self, 
        *,
        hidden_dim=256,
        n_lstm_layers=4,
        dropout=0.1,
        **kwargs,
    ):
        super().__init__()
        self.dropout = torch.nn.Dropout(dropout)
        
        self.nn = torch.nn.Sequential(
            building_block.ascii_embedding(hidden_dim//4),
            building_block.SeqCNN(hidden_dim//4),
            self.dropout,
            building_block.SeqLSTM(
                hidden_dim, hidden_dim, 
                rnn_layer=n_lstm_layers
            ),
            building_block.SeqAttentionSum(hidden_dim),
            self.dropout,
            torch.nn.Linear(hidden_dim,64),
            torch.nn.GELU(),
            torch.nn.Linear(64, 1),
        )
    def forward(self, aa_x):
        return self.nn(aa_x).squeeze(-1)

class ScalarRegression_Transformer_Model_for_AASeq(torch.nn.Module):
    def __init__(self,
        *,
        hidden_dim = 256,
        nlayers = 4,
        output_attentions=False,
        dropout = 0.1,
        **kwargs,
    ):
        """
        Model based on a transformer Architecture from 
        Huggingface's BertEncoder class.
        """
        super().__init__()

        self.dropout = torch.nn.Dropout(dropout)

        self.input_nn =  torch.nn.Sequential(
            building_block.ascii_embedding(hidden_dim),
        )

        self._output_attentions = output_attentions
        
        self.hidden_nn = building_block.HFace_Transformer_with_PositionalEncoder(
            hidden_dim, nlayers=nlayers, dropout=dropout,
            output_attentions=output_attentions
        )

        self.output_nn = torch.nn.Sequential(
            building_block.SeqAttentionSum(hidden_dim),
            torch.nn.PReLU(),
            self.dropout,
            torch.nn.Linear(hidden_dim, 1),
        )

    @property
    def output_attentions(self):
        return self._output_attentions

    @output_attentions.setter
    def output_attentions(self, val:bool):
        self._output_attentions = val
        self.hidden_nn.output_attentions = val

    def forward(self, aa_x):
        aa_x = self.dropout(self.input_nn(aa_x))

        aa_x = self.hidden_nn(aa_x)
        if self.output_attentions:
            self.attentions = aa_x[1]
        else:
            self.attentions = None
        aa_x = self.dropout(aa_x[0])

        return self.output_nn(aa_x).squeeze(1)

class ScalarRegression_ModelInterface_for_AASeq(ModelInterface):
    def __init__(self, 
        dropout=0.1,
        model_class:torch.nn.Module=ScalarRegression_LSTM_Model_for_AASeq, #model defined above
        device:str='gpu',
        **kwargs,
    ):
        super().__init__(device=device)
        self.build(
            model_class,
            dropout=dropout,
            **kwargs
        )
        self.loss_func = torch.nn.L1Loss() # for regression

    def _prepare_predict_data_df(self,
        precursor_df:pd.DataFrame,
    ):
        self._predict_column_in_df = 'target_value_pred'
        precursor_df[self._predict_column_in_df] = 0.
        self.predict_df = precursor_df

    def _get_features_from_batch_df(self, 
        batch_df: pd.DataFrame,
        **kwargs,
    ):
        aa_indices = self._as_tensor(
            get_ascii_indices(
                batch_df['sequence'].values.astype('U')
            ), 
            dtype=torch.long
        )

        return aa_indices

    def _get_targets_from_batch_df(self, 
        batch_df: pd.DataFrame,
        **kwargs
    ) -> torch.Tensor:
        return self._as_tensor(
            batch_df['target_value'].values, 
            dtype=torch.float32
        )

#### Building a RT model for only sequences based on `ScalarRegression_LSTM_Model_for_AASeq`

In [5]:
from peptdeep.model.rt import IRT_PEPTIDE_DF

IRT_PEPTIDE_DF=IRT_PEPTIDE_DF.copy()
IRT_PEPTIDE_DF['target_value'] = (
    IRT_PEPTIDE_DF.irt-IRT_PEPTIDE_DF.irt.min()
)/(IRT_PEPTIDE_DF.irt.max()-IRT_PEPTIDE_DF.irt.min())
model = ScalarRegression_ModelInterface_for_AASeq(
    model_class=ScalarRegression_LSTM_Model_for_AASeq
)
model.train(IRT_PEPTIDE_DF, epoch=20)
model.predict(IRT_PEPTIDE_DF)

,sequence,pep_name,irt,mods,mod_sites,nAA,target_value,target_value_pred
0,LGGNEQVTR,RT-pep a,-24.92,,,9,0.000000,0.000000
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,0.199488,0.270292
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,0.298671,0.312048
3,YILAGVENSK,RT-pep d,19.79,,,10,0.357909,0.375588
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,0.429315,0.511878
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,0.466699,0.567923
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0.537784,0.811736
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0.636728,0.687210
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0.764009,0.610351
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0.897775,0.827414


In [6]:
#hide
model.predict_mp(IRT_PEPTIDE_DF, mp_batch_size=2, process_num=2)

,sequence,pep_name,irt,mods,mod_sites,nAA,target_value,target_value_pred
0,LGGNEQVTR,RT-pep a,-24.92,,,9,0.000000,0.000000
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,0.199488,0.270292
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,0.298671,0.312048
3,YILAGVENSK,RT-pep d,19.79,,,10,0.357909,0.375588
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,0.429315,0.511878
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,0.466699,0.567923
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0.537784,0.811736
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0.636728,0.687210
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0.764009,0.610351
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0.897775,0.827414


#### Building a RT model for only sequences based on `ScalarRegression_Transformer_Model_for_AASeq`

In [7]:
from peptdeep.model.rt import IRT_PEPTIDE_DF

IRT_PEPTIDE_DF=IRT_PEPTIDE_DF.copy()
IRT_PEPTIDE_DF['target_value'] = (
    IRT_PEPTIDE_DF.irt-IRT_PEPTIDE_DF.irt.min()
)/(IRT_PEPTIDE_DF.irt.max()-IRT_PEPTIDE_DF.irt.min())
model = ScalarRegression_ModelInterface_for_AASeq(
    model_class=ScalarRegression_Transformer_Model_for_AASeq
)
model.train(IRT_PEPTIDE_DF, epoch=20)
model.predict(IRT_PEPTIDE_DF)

,sequence,pep_name,irt,mods,mod_sites,nAA,target_value,target_value_pred
0,LGGNEQVTR,RT-pep a,-24.92,,,9,0.000000,0.000000
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,0.199488,0.000000
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,0.298671,0.066876
3,YILAGVENSK,RT-pep d,19.79,,,10,0.357909,0.122096
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,0.429315,0.196392
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,0.466699,0.244039
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0.537784,0.112480
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0.636728,0.353008
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0.764009,0.590600
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0.897775,0.647294


# Binary Classification Models for a Given Amino Acid Sequence

In [8]:
#export
class BinaryClassification_LSTM_Model_for_AASeq(torch.nn.Module):
    def __init__(self, 
        *,
        hidden_dim=256,
        n_lstm_layers=4,
        dropout=0.1,
        **kwargs,
    ):
        super().__init__()
        
        self.nn = torch.nn.Sequential(
            ScalarRegression_LSTM_Model_for_AASeq(
                hidden_dim=hidden_dim, 
                input_dim=ASCII_NUM, 
                n_lstm_layers=n_lstm_layers,
                dropout=dropout,
            ),
            torch.nn.Sigmoid(),
        )
    def forward(self, aa_x):
        return self.nn(aa_x)

class BinaryClassification_Transformer_Model_for_AASeq(torch.nn.Module):
    def __init__(self,
        *,
        hidden_dim = 256,
        nlayers = 4,
        output_attentions=False,
        dropout = 0.1,
        **kwargs,
    ):
        """
        Model based on a transformer Architecture from 
        Huggingface's BertEncoder class.
        """
        super().__init__()

        self.nn =  torch.nn.Sequential(
            ScalarRegression_Transformer_Model_for_AASeq(
                nlayers=nlayers,
                input_dim=ASCII_NUM,
                hidden_dim=hidden_dim,
                output_attentions=output_attentions,
                dropout=dropout,
                **kwargs,
            ),
            torch.nn.Sigmoid(),
        )

    @property
    def output_attentions(self):
        return self._output_attentions

    @output_attentions.setter
    def output_attentions(self, val:bool):
        self._output_attentions = val
        self.hidden_nn.output_attentions = val

    def forward(self, aa_x):
        return self.nn(aa_x)

class BinaryClassification_ModelInterface_for_AASeq(ModelInterface):
    def __init__(self, 
        dropout=0.1,
        model_class:torch.nn.Module=BinaryClassification_LSTM_Model_for_AASeq, #model defined above
        device:str='gpu',
        **kwargs,
    ):
        """
        Class to predict retention times from precursor dataframes.
        """
        super().__init__(device=device)
        self.build(
            model_class,
            dropout=dropout,
            **kwargs
        )
        self.loss_func = torch.nn.BCELoss() # for binary classification

    def _prepare_predict_data_df(self,
        precursor_df:pd.DataFrame,
    ):
        self._predict_column_in_df = 'target_prob_pred'
        precursor_df[self._predict_column_in_df] = 0.
        self.predict_df = precursor_df

    def _get_features_from_batch_df(self, 
        batch_df: pd.DataFrame,
        **kwargs,
    ):
        aa_indices = self._as_tensor(
            get_ascii_indices(
                batch_df['sequence'].values.astype('U')
            ), dtype=torch.long
        )

        return aa_indices

    def _get_targets_from_batch_df(self, 
        batch_df: pd.DataFrame,
        **kwargs
    ) -> torch.Tensor:
        return self._as_tensor(
            batch_df['target_prob'].values, 
            dtype=torch.float32
        )

#### A sequence classification model using `BinaryClassification_LSTM_Model_for_AASeq`

In [9]:
from peptdeep.model.rt import IRT_PEPTIDE_DF

IRT_PEPTIDE_DF=IRT_PEPTIDE_DF.copy()
IRT_PEPTIDE_DF['target_prob'] = 0
IRT_PEPTIDE_DF.loc[:5,'target_prob']=1
model = BinaryClassification_ModelInterface_for_AASeq(
    model_class=BinaryClassification_LSTM_Model_for_AASeq
)
model.train(IRT_PEPTIDE_DF, epoch=20)
model.predict(IRT_PEPTIDE_DF)

,sequence,pep_name,irt,mods,mod_sites,nAA,target_prob,target_prob_pred
0,LGGNEQVTR,RT-pep a,-24.92,,,9,1,0.994553
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,1,0.993269
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,1,0.994685
3,YILAGVENSK,RT-pep d,19.79,,,10,1,0.994273
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,1,0.992981
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,1,0.992346
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0,0.373829
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0,0.381207
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0,0.387663
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0,0.380410


#### A sequence classification model using `BinaryClassification_LSTM_Model_for_AASeq`

In [10]:
from peptdeep.model.rt import IRT_PEPTIDE_DF

IRT_PEPTIDE_DF=IRT_PEPTIDE_DF.copy()
IRT_PEPTIDE_DF['target_prob'] = 0
IRT_PEPTIDE_DF.loc[:5,'target_prob']=1
model = BinaryClassification_ModelInterface_for_AASeq(
    model_class=BinaryClassification_Transformer_Model_for_AASeq
)
model.train(IRT_PEPTIDE_DF, epoch=10)
model.predict(IRT_PEPTIDE_DF)

,sequence,pep_name,irt,mods,mod_sites,nAA,target_prob,target_prob_pred
0,LGGNEQVTR,RT-pep a,-24.92,,,9,1,0.994852
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,1,0.986813
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,1,0.978719
3,YILAGVENSK,RT-pep d,19.79,,,10,1,0.993586
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,1,0.987523
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,1,0.986699
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0,0.014846
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0,0.021635
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0,0.010523
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0,0.009125


# Scalar Regression Models for Given Amino Acid Sequence and Site-specific PTMs

In [11]:
#export
class ScalarRegression_LSTM_Model_for_ModAASeq(torch.nn.Module):
    def __init__(self, 
        *,
        hidden_dim=256,
        n_lstm_layers=4,
        dropout=0.1,
        **kwargs,
    ):
        super().__init__()
        self.dropout = torch.nn.Dropout(dropout)

        self.encoder_nn = building_block.Encoder_AsciiAA_Mod_CNN_LSTM_AttnSum(
            hidden_dim,
            n_lstm_layers=n_lstm_layers,
        )
        self.output_nn = torch.nn.Sequential(
            self.dropout,
            torch.nn.Linear(hidden_dim,64),
            torch.nn.GELU(),
            torch.nn.Linear(64, 1),
        )
    def forward(self, aa_x, mod_x):
        x = self.encoder_nn(aa_x, mod_x)
        return self.output_nn(x).squeeze(-1)

class ScalarRegression_Transformer_Model_for_ModAASeq(torch.nn.Module):
    def __init__(self,
        *,
        hidden_dim = 256,
        nlayers = 4,
        output_attentions=False,
        dropout = 0.1,
        **kwargs,
    ):
        """
        Model based on a transformer Architecture from 
        Huggingface's BertEncoder class.
        """
        super().__init__()

        self.dropout = torch.nn.Dropout(dropout)

        self.input_nn = building_block.AA_Mod_Embedding(hidden_dim)

        self._output_attentions = output_attentions
        
        self.hidden_nn = building_block.HFace_Transformer_with_PositionalEncoder(
            hidden_dim, nlayers=nlayers, dropout=dropout,
            output_attentions=output_attentions
        )

        self.output_nn = torch.nn.Sequential(
            building_block.SeqAttentionSum(hidden_dim),
            torch.nn.PReLU(),
            self.dropout,
            torch.nn.Linear(hidden_dim, 1),
        )

    @property
    def output_attentions(self):
        return self._output_attentions

    @output_attentions.setter
    def output_attentions(self, val:bool):
        self._output_attentions = val
        self.hidden_nn.output_attentions = val

    def forward(self, 
        aa_indices, 
        mod_x,
    ):
        x = self.dropout(self.input_nn(
            aa_indices, mod_x
        ))

        hidden_x = self.hidden_nn(x)
        if self.output_attentions:
            self.attentions = hidden_x[1]
        else:
            self.attentions = None
        x = self.dropout(hidden_x[0]+x*0.2)

        return self.output_nn(x).squeeze(1)

class ScalarRegression_ModelInterface_for_ModAASeq(ModelInterface):
    def __init__(self, 
        dropout=0.1,
        model_class:torch.nn.Module=ScalarRegression_LSTM_Model_for_ModAASeq, #model defined above
        device:str='gpu',
        **kwargs,
    ):
        super().__init__(device=device)
        self.build(
            model_class,
            dropout=dropout,
            **kwargs
        )
        self.loss_func = torch.nn.L1Loss() # for regression

    def _prepare_predict_data_df(self,
        precursor_df:pd.DataFrame,
    ):
        self._predict_column_in_df = 'target_value_pred'
        precursor_df[self._predict_column_in_df] = 0.
        self.predict_df = precursor_df

    def _get_features_from_batch_df(self, 
        batch_df: pd.DataFrame,
        **kwargs,
    ):
        aa_indices = self._as_tensor(
            get_ascii_indices(
                batch_df['sequence'].values.astype('U')
            ),
            dtype=torch.long
        )
        
        mod_x = self._as_tensor(
            get_batch_mod_feature(
                batch_df
            )
        )

        return aa_indices, mod_x

    def _get_targets_from_batch_df(self, 
        batch_df: pd.DataFrame,
        **kwargs
    ) -> torch.Tensor:
        return self._as_tensor(
            batch_df['target_value'].values, 
            dtype=torch.float32
        )

#### Scalar regression model (RT) with modified AA sequences using `ScalarRegression_LSTM_Model_for_ModAASeq`

In [12]:
from peptdeep.model.rt import IRT_PEPTIDE_DF

IRT_PEPTIDE_DF=IRT_PEPTIDE_DF.copy()
IRT_PEPTIDE_DF['target_value'] = (
    IRT_PEPTIDE_DF.irt-IRT_PEPTIDE_DF.irt.min()
)/(IRT_PEPTIDE_DF.irt.max()-IRT_PEPTIDE_DF.irt.min())
model = ScalarRegression_ModelInterface_for_ModAASeq(
    model_class=ScalarRegression_LSTM_Model_for_ModAASeq
)
model.train(IRT_PEPTIDE_DF, epoch=20)
model.predict(IRT_PEPTIDE_DF)

,sequence,pep_name,irt,mods,mod_sites,nAA,target_value,target_value_pred
0,LGGNEQVTR,RT-pep a,-24.92,,,9,0.000000,0.002815
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,0.199488,0.176670
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,0.298671,0.240738
3,YILAGVENSK,RT-pep d,19.79,,,10,0.357909,0.358938
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,0.429315,0.320344
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,0.466699,0.405891
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0.537784,0.569023
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0.636728,0.555376
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0.764009,0.823821
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0.897775,0.939385


#### Scalar regression model (RT) with modified AA sequences using `ScalarRegression_Transformer_Model_for_ModAASeq`

In [13]:
from peptdeep.model.rt import IRT_PEPTIDE_DF

IRT_PEPTIDE_DF=IRT_PEPTIDE_DF.copy()
IRT_PEPTIDE_DF['target_value'] = (
    IRT_PEPTIDE_DF.irt-IRT_PEPTIDE_DF.irt.min()
)/(IRT_PEPTIDE_DF.irt.max()-IRT_PEPTIDE_DF.irt.min())
model = ScalarRegression_ModelInterface_for_ModAASeq(
    model_class=ScalarRegression_Transformer_Model_for_ModAASeq
)
model.train(IRT_PEPTIDE_DF, epoch=10)
model.predict(IRT_PEPTIDE_DF)

,sequence,pep_name,irt,mods,mod_sites,nAA,target_value,target_value_pred
0,LGGNEQVTR,RT-pep a,-24.92,,,9,0.000000,0.000000
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,0.199488,0.093673
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,0.298671,0.285840
3,YILAGVENSK,RT-pep d,19.79,,,10,0.357909,0.372704
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,0.429315,0.433030
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,0.466699,0.371391
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0.537784,0.618134
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0.636728,0.665806
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0.764009,0.814970
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0.897775,0.890673


# Binary Classification Models for Given Amino Acid Sequence and Site-specific PTMs

In [14]:
#export
class BinaryClassification_LSTM_Model_for_ModAASeq(torch.nn.Module):
    def __init__(self, 
        *,
        hidden_dim=256,
        n_lstm_layers=4,
        dropout=0.1,
        **kwargs,
    ):
        super().__init__()
        self.nn = ScalarRegression_LSTM_Model_for_ModAASeq(
            hidden_dim=hidden_dim,
            n_lstm_layers=n_lstm_layers,
            dropout=dropout
        )

    def forward(self, aa_x, mod_x):
        return torch.sigmoid(self.nn(aa_x, mod_x))

class BinaryClassification_Transformer_Model_for_ModAASeq(torch.nn.Module):
    def __init__(self,
        *,
        hidden_dim = 256,
        nlayers = 4,
        output_attentions=False,
        dropout = 0.1,
        **kwargs,
    ):
        """
        Model based on a transformer Architecture from 
        Huggingface's BertEncoder class.
        """
        super().__init__()
        self.nn = ScalarRegression_Transformer_Model_for_ModAASeq(
            nlayers=nlayers,
            hidden_dim=hidden_dim,
            output_attentions=output_attentions,
            dropout=dropout,
            **kwargs
        )

    @property
    def output_attentions(self):
        return self._output_attentions

    @output_attentions.setter
    def output_attentions(self, val:bool):
        self._output_attentions = val
        self.hidden_nn.output_attentions = val

    def forward(self, 
        aa_indices, 
        mod_x,
    ):
        return torch.sigmoid(self.nn(aa_indices, mod_x))

class BinaryClassification_ModelInterface_for_ModAASeq(ModelInterface):
    def __init__(self, 
        dropout=0.1,
        model_class:torch.nn.Module=BinaryClassification_LSTM_Model_for_ModAASeq, #model defined above
        device:str='gpu',
        **kwargs,
    ):
        super().__init__(device=device)
        self.build(
            model_class,
            dropout=dropout,
            **kwargs
        )
        self.loss_func = torch.nn.BCELoss() # for regression

    def _prepare_predict_data_df(self,
        precursor_df:pd.DataFrame,
    ):
        self._predict_column_in_df = 'target_prob_pred'
        precursor_df[self._predict_column_in_df] = 0.
        self.predict_df = precursor_df

    def _get_features_from_batch_df(self, 
        batch_df: pd.DataFrame,
    ):
        aa_indices = self._as_tensor(
            get_ascii_indices(
                batch_df['sequence'].values.astype('U')
            ),
            dtype=torch.long
        )
        
        mod_x = self._as_tensor(
            get_batch_mod_feature(
                batch_df
            )
        )

        return aa_indices, mod_x

    def _get_targets_from_batch_df(self, 
        batch_df: pd.DataFrame,
        **kwargs
    ) -> torch.Tensor:
        return self._as_tensor(
            batch_df['target_prob'].values, 
            dtype=torch.float32
        )

#### Scalar regression model (RT) with modified AA sequences using `BinaryClassification_LSTM_Model_for_ModAASeq`

In [15]:
from peptdeep.model.rt import IRT_PEPTIDE_DF

IRT_PEPTIDE_DF=IRT_PEPTIDE_DF.copy()
IRT_PEPTIDE_DF['target_prob'] = 0
IRT_PEPTIDE_DF.loc[:5,'target_prob']=1
model = BinaryClassification_ModelInterface_for_ModAASeq(
    model_class=BinaryClassification_LSTM_Model_for_ModAASeq
)
model.train(IRT_PEPTIDE_DF, epoch=20)
model.predict(IRT_PEPTIDE_DF)

,sequence,pep_name,irt,mods,mod_sites,nAA,target_prob,target_prob_pred
0,LGGNEQVTR,RT-pep a,-24.92,,,9,1,0.993448
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,1,0.991897
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,1,0.993917
3,YILAGVENSK,RT-pep d,19.79,,,10,1,0.993438
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,1,0.991815
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,1,0.992453
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0,0.409278
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0,0.416397
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0,0.412872
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0,0.411099


#### Scalar regression model (RT) with modified AA sequences using `BinaryClassification_Transformer_Model_for_ModAASeq`

In [16]:
from peptdeep.model.rt import IRT_PEPTIDE_DF

IRT_PEPTIDE_DF=IRT_PEPTIDE_DF.copy()
IRT_PEPTIDE_DF['target_prob'] = 0
IRT_PEPTIDE_DF.loc[:5,'target_prob']=1
model = BinaryClassification_ModelInterface_for_ModAASeq(
    model_class=BinaryClassification_Transformer_Model_for_ModAASeq
)
model.train(IRT_PEPTIDE_DF, epoch=10)
model.predict(IRT_PEPTIDE_DF)

,sequence,pep_name,irt,mods,mod_sites,nAA,target_prob,target_prob_pred
0,LGGNEQVTR,RT-pep a,-24.92,,,9,1,0.993029
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,1,0.982783
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,1,0.983086
3,YILAGVENSK,RT-pep d,19.79,,,10,1,0.992445
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,1,0.984391
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,1,0.977889
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0,0.036614
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0,0.063267
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0,0.011384
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0,0.009429


In [17]:
#hide
model.predict_mp(IRT_PEPTIDE_DF, mp_batch_size=2, process_num=2)

,sequence,pep_name,irt,mods,mod_sites,nAA,target_prob,target_prob_pred
0,LGGNEQVTR,RT-pep a,-24.92,,,9,1,0.993029
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,1,0.982783
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,1,0.983086
3,YILAGVENSK,RT-pep d,19.79,,,10,1,0.992445
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,1,0.984391
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,1,0.977889
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0,0.036614
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0,0.063267
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0,0.011384
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0,0.009429
